<a href="https://colab.research.google.com/github/Pathway2008/CarPrice/blob/main/vsmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1. All data 예측 vs 그룹화 vs 개별 - 성능비교 차이없음, xgb-lgb model 선택

2. 배터리용량이 isna vs not nan 값 성능비교 14,15번 차이있음, 4번 차이없음 - 가격의 루트를 씌워서 범위를 줄이는 건? -효과없음
'''

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [89]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/sample_submission.csv')

In [90]:
ordinal_features = ['제조사', '차량상태', '구동방식','사고이력',  '모델']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [6]:
model_mapping = dict(enumerate(le.classes_))
model_mapping

{0: 'EV6',
 1: 'ID4',
 2: 'ION5',
 3: 'ION6',
 4: 'IONIQ',
 5: 'KNE',
 6: 'M3',
 7: 'MS',
 8: 'MX',
 9: 'MY',
 10: 'Niro',
 11: 'Q4eT',
 12: 'RSeTGT',
 13: 'Soul',
 14: 'Tay',
 15: 'TayCT',
 16: 'TayGTS',
 17: 'eT',
 18: 'i3',
 19: 'i5',
 20: 'iX'}

In [8]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [16]:
!pip install --upgrade scikit-learn xgboost catboost lightgbm

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split, cross_val_score,StratifiedKFold
from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt

In [30]:
# All_data
X_train_all = train.drop(columns=['ID', '가격(백만원)'])
y_train_all = train['가격(백만원)']
model_column_1 = train['모델']

# Stratified Split (모델 컬럼의 비율 유지)
X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
    X_train_all, y_train_all, test_size=0.2, random_state=42, stratify=model_column_1)

In [31]:
class SklearnCompatibleRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 호환 가능한 모델 생성
xgb_model = SklearnCompatibleRegressor(XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42))
cat_model = SklearnCompatibleRegressor(CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=42, verbose=0))
lgb_model = SklearnCompatibleRegressor(LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=42))

# Voting Regressors 정의
voting_models = {
    'XGB+Cat': VotingRegressor([('xgb', xgb_model), ('cat', cat_model)]),
    'XGB+LGB': VotingRegressor([('xgb', xgb_model), ('lgb', lgb_model)]),
    'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)]),
    'XGB+Cat+LGB': VotingRegressor([('xgb', xgb_model), ('cat', cat_model), ('lgb', lgb_model)])
}

In [32]:
xgb_model.fit(X_train_stratified, y_train_stratified)
lgb_model.fit(X_train_stratified, y_train_stratified)
cat_model.fit(X_train_stratified, y_train_stratified)


# 모델 학습
for name, voting_model in voting_models.items():
    voting_model.fit(X_train_stratified, y_train_stratified)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 406
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.370517
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 406
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.370517
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000476 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 406
[LightGBM] [Info] Number of data points in the train set: 599

In [33]:
y_pred_xgb = xgb_model.predict(X_valid_stratified)
y_pred_lgb = lgb_model.predict(X_valid_stratified)
y_pred_cat = cat_model.predict(X_valid_stratified)

In [37]:
result_df = pd.DataFrame(X_valid_stratified)
result_df['가격(백만원)'] = y_valid_stratified.values
result_df['XGB_Pred'] = y_pred_xgb
result_df['CatBoost_Pred'] = y_pred_cat
result_df['LGB_Pred'] = y_pred_lgb
# 각 Voting 모델에 대한 예측
for name, voting_model in voting_models.items():
    result_df[f'{name}_Pred'] = voting_model.predict(X_valid_stratified)

In [38]:
result_df

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원),XGB_Pred,CatBoost_Pred,LGB_Pred,XGB+Cat_Pred,XGB+LGB_Pred,Cat+LGB_Pred,XGB+Cat+LGB_Pred
116,1,19,0,NaN,0,4692,8,1,0,65.94,64.768555,64.899842,64.864000,64.834198,64.816277,64.881921,64.844132
2956,2,3,0,NaN,0,1746,10,0,0,38.04,38.934402,38.706357,38.654218,38.820380,38.794310,38.680288,38.764993
1149,6,1,2,57.861,0,189751,1,0,0,39.36,40.334442,40.150482,40.089024,40.242462,40.211733,40.119753,40.191316
6962,3,13,2,46.169,1,182674,2,0,0,22.29,22.467823,22.613747,22.566297,22.540785,22.517060,22.590022,22.549289
3442,3,10,0,90.000,1,2708,9,0,0,23.79,24.721424,24.216635,24.456314,24.469030,24.588869,24.336475,24.464791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,2,5,2,68.488,1,178121,3,0,0,28.26,27.798649,27.563470,27.924035,27.681059,27.861342,27.743752,27.762051
2030,1,19,0,NaN,0,5237,7,0,0,64.61,65.074074,65.071018,65.352219,65.072546,65.213146,65.211619,65.165770
5803,2,3,2,60.585,0,85766,4,0,0,37.29,37.710941,37.609106,37.381790,37.660024,37.546366,37.495448,37.567279
6243,5,7,0,99.800,0,1597,8,0,0,64.58,64.071091,64.320349,64.169263,64.195720,64.120177,64.244806,64.186901


In [39]:
result_df['xgb-real'] =( (result_df['가격(백만원)'] - result_df['XGB_Pred'])**2)**0.5
result_df['cat-real'] =( (result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
result_df['lgb-real'] =( (result_df['가격(백만원)'] - result_df['LGB_Pred'])**2)**0.5
result_df['xgb_cat-real'] = ( (result_df['가격(백만원)'] - result_df['XGB+Cat_Pred'])**2)**0.5
result_df['xgb_lgb-real'] = ( (result_df['가격(백만원)'] - result_df['XGB+LGB_Pred'])**2)**0.5
result_df['cat_lgb-real'] = ( (result_df['가격(백만원)'] - result_df['Cat+LGB_Pred'])**2)**0.5
result_df['xgb_cat_lgb-real'] = ( (result_df['가격(백만원)'] - result_df['XGB+Cat+LGB_Pred'])**2)**0.5

In [41]:
model_stats = result_df.groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats

xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.532396  0.321014  0.539011  0.362712  0.530005  0.328702     0.531353   
1   0.553992  0.298476  0.547958  0.358690  0.552602  0.320050     0.538073   
2   0.262312  0.187226  0.276192  0.225977  0.285691  0.197186     0.262207   
3   0.274482  0.183291  0.311935  0.166891  0.253990  0.192122     0.286018   
4   4.956991  3.013849  5.443629  2.645631  5.300255  2.692699     5.111009   
5   0.430788  0.333622  0.444684  0.343078  0.460297  0.359549     0.422400   
6   0.346553  0.260023  0.405747  0.308711  0.413545  0.257089     0.356250   
7   0.595063  0.389178  0.645789  0.432230  0.588875  0.377349     0.609949   
8   0.582060  0.423472  0.609787  0.404066  0.497015  0.352360     0.584987   
9   0.517496  0.332451  0.771991  0.446617  0.514745  0.319940     0.625050   
10  0.493632  0.334383  0.536799  0.389049  0.535521  0.348641     0.507624   
11  0.300262  0.200748  0.357183  0.278502  0.296237  0.204086     0.311474   
12  0.330449  0.275560  0.398375  0.448620  0.316267  0.259053     0.326479   
13  0.503948  0.296037  0.639339  0.406771  0.510969  0.288197     0.531545   
14  2.331786  2.229626  2.569016  1.881062  2.381924  2.199561     2.416625   
15  3.039258  2.666140  2.972052  2.377561  2.997605  2.630370     2.989477   
16  0.422363  0.417103  0.651430  0.478605  0.441906  0.322953     0.462854   
17  0.446991  0.312507  0.476920  0.352698  0.412585  0.321853     0.441807   
18  0.299863  0.162450  0.433346  0.269990  0.289202  0.178518     0.338063   
19  0.514918  0.325792  0.606182  0.411889  0.544812  0.343573     0.532406   
20  0.477464  0.316277  0.502168  0.264736  0.469978  0.300705     0.477748   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.331747     0.529412  0.318788     0.532852  0.335603         0.530845   
1   0.320253     0.550314  0.301296     0.539435  0.329939         0.539589   
2   0.197311     0.267918  0.189710     0.264241  0.205401         0.260515   
3   0.160987     0.261583  0.184559     0.277530  0.160149         0.273272   
4   2.794778     5.064912  2.848963     5.354781  2.592816         5.161125   
5   0.325148     0.444321  0.342366     0.430802  0.341023         0.425363   
6   0.268246     0.367765  0.251168     0.356728  0.277792         0.344458   
7   0.351769     0.587461  0.367324     0.596340  0.370332         0.594407   
8   0.384807     0.530170  0.356143     0.542584  0.336720         0.548951   
9   0.380257     0.513849  0.313368     0.629396  0.367373         0.582853   
10  0.338313     0.510024  0.336686     0.521818  0.349020         0.508711   
11  0.234595     0.295858  0.197929     0.311903  0.235599         0.303527   
12  0.294186     0.316552  0.257506     0.332100  0.302185         0.315925   
13  0.331109     0.505442  0.283319     0.532346  0.321457         0.512590   
14  2.011654     2.348593  2.200440     2.456848  2.000791         2.398587   
15  2.492745     3.017760  2.621139     2.967838  2.488348         2.985420   
16  0.335494     0.413332  0.315397     0.465376  0.343522         0.435843   
17  0.315908     0.416288  0.293322     0.410251  0.283889         0.413951   
18  0.199071     0.288830  0.167158     0.324546  0.204286         0.304776   
19  0.340346     0.526157  0.328881     0.549620  0.341917         0.530065   
20  0.276459     0.467163  0.304802     0.477924  0.263192         0.471433   

              
         std  
모델            
0   0.324834  
1   0.314310  
2   0.196469  
3   0.164625  
4   2.728930  
5   0.336830  
6   0.269675  
7   0.348925  
8   0.347597  
9   0.352178  
10  0.337748  


In [42]:
# 'mean' 열만 필터링
mean_columns = [col for col in model_stats.columns if col[1] == 'mean']

# 스타일링 함수 정의
def highlight_minimum(s):
    is_min = s == s.min()  # 최소값 여부 확인
    return ['background-color: yellow' if v else '' for v in is_min]

# 스타일 적용
styled_model_stats = model_stats.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

styled_model_stats

In [43]:
model_stats_notnan = result_df[~result_df['배터리용량'].isna()].groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats_isnan = result_df[result_df['배터리용량'].isna()].groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])

In [45]:
model_stats_notnan

xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.490157  0.289160  0.552067  0.433390  0.520250  0.336232     0.514167   
1   0.544244  0.299138  0.558403  0.386271  0.542591  0.330506     0.531173   
2   0.261245  0.190810  0.261567  0.247193  0.270080  0.197122     0.255482   
3   0.270903  0.160602  0.300792  0.160710  0.258998  0.187874     0.276179   
4   5.001249  2.999691  5.673192  2.650248  5.535954  2.759041     5.241033   
5   0.426477  0.356702  0.451929  0.369640  0.477923  0.392115     0.418113   
6   0.293045  0.243768  0.432033  0.327015  0.413281  0.245360     0.335346   
7   0.662519  0.389084  0.611376  0.424088  0.637270  0.382668     0.636014   
8   0.579803  0.431623  0.608736  0.399110  0.505019  0.361337     0.581819   
9   0.545221  0.363397  0.859673  0.441128  0.527707  0.300242     0.684729   
10  0.459631  0.339262  0.497039  0.376170  0.500667  0.356686     0.469270   
11  0.346174  0.221743  0.414066  0.319535  0.345905  0.223179     0.357375   
12  0.447679  0.361585  0.612370  0.736322  0.327465  0.253768     0.459958   
13  0.489485  0.285328  0.634611  0.415605  0.507445  0.291924     0.517113   
14  1.519415  2.197655  1.634064  1.821831  1.538119  2.275685     1.522695   
15  1.571541  1.610153  1.517711  1.369084  1.475848  1.383337     1.519418   
16  0.386232  0.443326  0.759218  0.535893  0.423404  0.323509     0.496666   
17  0.433509  0.308296  0.417750  0.286380  0.433626  0.337510     0.401339   
18  0.309034  0.165320  0.457109  0.267406  0.291644  0.184987     0.354961   
19  0.509995  0.319486  0.611744  0.430504  0.527308  0.341352     0.529905   
20  0.524532  0.416416  0.468140  0.167651  0.529542  0.403171     0.475423   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.361073     0.505203  0.298256     0.531884  0.381496         0.516007   
1   0.342350     0.540203  0.310853     0.535016  0.349412         0.531930   
2   0.204470     0.258603  0.190844     0.248320  0.213456         0.249035   
3   0.152448     0.261243  0.171220     0.272427  0.158919         0.267388   
4   2.814578     5.183654  2.901072     5.604573  2.603670         5.322052   
5   0.352714     0.450897  0.369120     0.434110  0.370799         0.423035   
6   0.275147     0.331575  0.233483     0.341142  0.279939         0.309672   
7   0.346163     0.644416  0.369531     0.619214  0.370140         0.631869   
8   0.389801     0.532720  0.362310     0.551176  0.334159         0.553728   
9   0.388930     0.532904  0.323614     0.675943  0.365319         0.624083   
10  0.337055     0.474713  0.344240     0.481728  0.350630         0.469931   
11  0.264967     0.342008  0.220052     0.358872  0.271375         0.349487   
12  0.335686     0.384920  0.285719     0.426651  0.362304         0.398530   
13  0.328505     0.496051  0.281798     0.522216  0.326547         0.500617   
14  1.986608     1.518462  2.234154     1.554169  2.027860         1.516744   
15  1.472761     1.522648  1.479615     1.470306  1.353179         1.494353   
16  0.359828     0.379904  0.317849     0.504466  0.376298         0.444397   
17  0.288943     0.419178  0.302126     0.387537  0.280031         0.392385   
18  0.199250     0.293893  0.172904     0.338771  0.206111         0.316313   
19  0.342941     0.514255  0.324683     0.539637  0.347017         0.521889   
20  0.296185     0.507272  0.424191     0.482200  0.290453         0.482938   

              
         std  
모델            
0   0.345240  
1   0.331245  
2   0.201788  
3   0.158076  
4   2.770548  
5   0.367731  
6   0.270425  
7   0.349060  
8   0.347381  
9   0.359072  
10  0.341663  


In [46]:
model_stats_isnan

xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.539771  0.327834  0.536732  0.352946  0.531708  0.330091     0.534354   
1   0.569340  0.300005  0.531514  0.313683  0.568364  0.305694     0.548938   
2   0.263580  0.185913  0.293559  0.200394  0.304228  0.198779     0.270194   
3   0.278674  0.209117  0.324988  0.175296  0.248124  0.199575     0.297543   
4   4.824216  3.294472  4.754938  2.710467  4.593157  2.544745     4.720936   
5   0.438133  0.296514  0.432341  0.298687  0.430268  0.300656     0.429705   
6   0.408293  0.269102  0.375417  0.289534  0.413849  0.274908     0.380370   
7   0.325240  0.258486  0.783441  0.457634  0.395294  0.296180     0.505687   
8   0.596887  0.395790  0.616693  0.469328  0.444417  0.305090     0.605807   
9   0.468647  0.270852  0.617503  0.423001  0.491906  0.358623     0.519900   
10  0.668867  0.251661  0.741716  0.404797  0.715154  0.242346     0.705291   
11  0.237133  0.148964  0.278970  0.187402  0.227943  0.152850     0.248360   
12  0.294684  0.235819  0.333089  0.293702  0.312851  0.262695     0.285757   
13  0.577377  0.348816  0.663347  0.373033  0.528861  0.279019     0.604813   
14  3.469104  1.746107  3.877949  0.965458  3.563250  1.431231     3.668126   
15  5.668919  2.102439  5.577745  1.326309  5.724085  2.067200     5.623332   
16  0.494625  0.356369  0.435855  0.217565  0.478910  0.325225     0.395230   
17  0.506696  0.335757  0.738959  0.492129  0.319400  0.227278     0.621022   
18  0.229551  0.124119  0.251161  0.227154  0.270482  0.124171     0.208514   
19  0.541425  0.370780  0.576230  0.304093  0.639062  0.353860     0.545870   
20  0.468205  0.296170  0.508862  0.280486  0.458260  0.279032     0.478205   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.329382     0.533639  0.324325     0.533021  0.330360         0.533435   
1   0.285236     0.566232  0.288175     0.546393  0.300252         0.551647   
2   0.191398     0.278981  0.190796     0.283147  0.197089         0.274149   
3   0.171971     0.261982  0.201611     0.283508  0.163694         0.280164   
4   2.916448     4.708686  2.876317     4.605407  2.604148         4.678343   
5   0.278133     0.433119  0.297608     0.425166  0.289884         0.429330   
6   0.263353     0.409523  0.268637     0.374713  0.279711         0.384595   
7   0.371469     0.359640  0.264686     0.504845  0.374112         0.444559   
8   0.378363     0.513409  0.338190     0.486121  0.375342         0.517559   
9   0.348740     0.480277  0.299178     0.547384  0.365171         0.510209   
10  0.279509     0.692010  0.227323     0.728435  0.265614         0.708579   
11  0.169298     0.232401  0.142947     0.247321  0.157167         0.240333   
12  0.270480     0.295694  0.247117     0.303254  0.278497         0.290723   
13  0.348008     0.553119  0.297767     0.583776  0.301081         0.573380   
14  1.245100     3.510776  1.549920     3.720599  1.074612         3.633167   
15  1.609639     5.696502  2.012561     5.650915  1.649140         5.656917   
16  0.274875     0.480188  0.305722     0.387196  0.255354         0.418737   
17  0.376308     0.403487  0.260466     0.510842  0.289212         0.509460   
18  0.150452     0.250016  0.113620     0.215493  0.159555         0.216329   
19  0.339209     0.590244  0.357315     0.603374  0.320526         0.574086   
20  0.275011     0.459273  0.279485     0.477083  0.260102         0.469169   

              
         std  
모델            
0   0.323987  
1   0.288699  
2   0.192260  
3   0.174054  
4   2.750046  
5   0.283036  
6   0.268397  
7   0.320380  
8   0.375178  
9   0.335737  
10  0.239323  


In [55]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Function to calculate RMSE for each fold
def calculate_rmse(model, X, y):
    y_pred = model.predict(X)
    return np.sqrt(mean_squared_error(y, y_pred))

# Initialize dictionaries to store RMSE results
rmse_results = {
    'XGB': [],
    'CatBoost': [],
    'LGB': [],
    'XGB+Cat': [],
    'XGB+LGB': [],
    'Cat+LGB': [],
    'XGB+Cat+LGB': []
}

# Perform Stratified K-Fold cross-validation
for train_index, valid_index in skf.split(X_train_all, model_column_1):
    X_train_kfold, X_valid_kfold = X_train_all.iloc[train_index], X_train_all.iloc[valid_index]
    y_train_kfold, y_valid_kfold = y_train_all.iloc[train_index], y_train_all.iloc[valid_index]

    # Fit individual models
    xgb_model.fit(X_train_kfold, y_train_kfold)
    cat_model.fit(X_train_kfold, y_train_kfold)
    lgb_model.fit(X_train_kfold, y_train_kfold)

    # Fit Voting models
    for name, voting_model in voting_models.items():
        voting_model.fit(X_train_kfold, y_train_kfold)

    # Calculate RMSE for each model
    rmse_results['XGB'].append(calculate_rmse(xgb_model, X_valid_kfold, y_valid_kfold))
    rmse_results['CatBoost'].append(calculate_rmse(cat_model, X_valid_kfold, y_valid_kfold))
    rmse_results['LGB'].append(calculate_rmse(lgb_model, X_valid_kfold, y_valid_kfold))

    for name, voting_model in voting_models.items():
        rmse_results[name].append(calculate_rmse(voting_model, X_valid_kfold, y_valid_kfold))

# Calculate average RMSE for each model
average_rmse = {model: np.mean(rmse_results[model]) for model in rmse_results}

# Output the average RMSE for each model
print("Average RMSE for each model:")
for model, rmse in average_rmse.items():
    print(f"{model}: {rmse:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.341322
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.341322
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000703 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4

In [57]:
#10 times
import numpy as np

# List to store the results for each iteration
results = []

# Repeat the process 10 times with different random states
for i in range(1, 11):
    random_state = i  # Different random_state for each iteration

    # Stratified Split with a different random state for each iteration
    X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
        X_train_all, y_train_all, test_size=0.2, random_state=random_state, stratify=model_column_1)

    # Re-initialize models with random_state
    xgb_model = SklearnCompatibleRegressor(XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=random_state))
    cat_model = SklearnCompatibleRegressor(CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=random_state, verbose=0))
    lgb_model = SklearnCompatibleRegressor(LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=random_state))

    # Fit models
    xgb_model.fit(X_train_stratified, y_train_stratified)
    lgb_model.fit(X_train_stratified, y_train_stratified)
    cat_model.fit(X_train_stratified, y_train_stratified)

    # Voting Regressors
    voting_models = {
        'XGB+Cat': VotingRegressor([('xgb', xgb_model), ('cat', cat_model)]),
        'XGB+LGB': VotingRegressor([('xgb', xgb_model), ('lgb', lgb_model)]),
        'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)]),
        'XGB+Cat+LGB': VotingRegressor([('xgb', xgb_model), ('cat', cat_model), ('lgb', lgb_model)]),
    }

    # Train voting models
    for name, voting_model in voting_models.items():
        voting_model.fit(X_train_stratified, y_train_stratified)

    # Predict using individual models and voting models
    y_pred_xgb = xgb_model.predict(X_valid_stratified)
    y_pred_lgb = lgb_model.predict(X_valid_stratified)
    y_pred_cat = cat_model.predict(X_valid_stratified)

    result_df = pd.DataFrame(X_valid_stratified)
    result_df['가격(백만원)'] = y_valid_stratified.values
    result_df['XGB_Pred'] = y_pred_xgb
    result_df['CatBoost_Pred'] = y_pred_cat
    result_df['LGB_Pred'] = y_pred_lgb

    # Add predictions from voting models
    for name, voting_model in voting_models.items():
        result_df[f'{name}_Pred'] = voting_model.predict(X_valid_stratified)

    # Calculate errors for each model
    result_df['xgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB_Pred'])**2)**0.5
    result_df['cat-real'] = ((result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
    result_df['lgb-real'] = ((result_df['가격(백만원)'] - result_df['LGB_Pred'])**2)**0.5
    result_df['xgb_cat-real'] = ((result_df['가격(백만원)'] - result_df['XGB+Cat_Pred'])**2)**0.5
    result_df['xgb_lgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB+LGB_Pred'])**2)**0.5
    result_df['cat_lgb-real'] = ((result_df['가격(백만원)'] - result_df['Cat+LGB_Pred'])**2)**0.5
    result_df['xgb_cat_lgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB+Cat+LGB_Pred'])**2)**0.5

    # Append the result_df to the results list
    results.append(result_df)

# Average the results across all 10 runs
final_result_df = pd.concat(results).groupby(level=0).mean()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000499 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.348574
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000481 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.348574
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000931 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4

In [58]:
final_result_df

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원),...,XGB+LGB_Pred,Cat+LGB_Pred,XGB+Cat+LGB_Pred,xgb-real,cat-real,lgb-real,xgb_cat-real,xgb_lgb-real,cat_lgb-real,xgb_cat_lgb-real
0,4.0,16.0,1.0,86.077,0.0,13642.0,0.0,0.0,2.0,159.66,...,159.452811,158.500643,158.865663,0.064297,1.968634,0.350080,1.016465,0.207189,1.159357,0.794337
1,3.0,10.0,1.0,56.000,1.0,10199.0,6.0,0.0,0.0,28.01,...,28.093387,27.778223,27.933577,0.234286,0.396043,0.067511,0.080879,0.083387,0.231777,0.076423
2,0.0,17.0,0.0,91.200,0.0,2361.0,7.0,0.0,0.0,66.27,...,65.684163,65.657592,65.629422,0.696918,0.750061,0.474756,0.723489,0.585837,0.612408,0.640578
3,0.0,12.0,1.0,NaN,0.0,21683.0,3.0,0.0,0.0,99.16,...,99.146433,99.237664,99.230972,0.057587,0.386060,0.084720,0.191163,0.039077,0.155820,0.102635
4,1.0,19.0,2.0,61.018,0.0,178205.0,1.0,0.0,0.0,62.02,...,62.808281,62.656828,62.713414,0.806586,0.503680,0.769976,0.655133,0.788281,0.636828,0.693414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7490,0.0,11.0,0.0,NaN,0.0,5794.0,7.0,0.0,0.0,59.95,...,59.809932,59.727941,59.761174,0.122359,0.286342,0.157776,0.204351,0.140068,0.222059,0.188826
7492,2.0,2.0,0.0,NaN,0.0,3773.0,10.0,0.0,0.0,35.95,...,35.686972,35.760550,35.738362,0.256015,0.108858,0.270041,0.182436,0.263028,0.189450,0.211638
7493,1.0,18.0,2.0,46.000,2.0,135411.0,2.0,0.0,0.0,23.40,...,23.257011,23.048728,23.116047,0.149314,0.565881,0.136663,0.357597,0.142989,0.351272,0.283953
7494,4.0,15.0,0.0,NaN,0.0,1363.0,2.0,0.0,0.0,120.00,...,127.902322,127.116374,127.422365,8.034348,6.462450,7.770297,7.248399,7.902322,7.116374,7.422365


In [59]:
model_stats = final_result_df.groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats

xgb-real            cat-real            lgb-real           xgb_cat-real  \
          mean       std      mean       std      mean       std         mean   
모델                                                                              
0.0   0.522647  0.325684  0.522256  0.346147  0.510335  0.326213     0.516423   
1.0   0.526701  0.319528  0.547686  0.344217  0.543544  0.331596     0.530495   
2.0   0.286669  0.188150  0.293248  0.262581  0.299188  0.212723     0.273712   
3.0   0.284784  0.180152  0.345334  0.156972  0.292595  0.184525     0.307649   
4.0   5.062470  3.174913  5.048608  2.833002  5.076072  3.035484     4.994963   
5.0   0.475190  0.319574  0.498064  0.354435  0.488410  0.327599     0.475645   
6.0   0.337112  0.272351  0.395530  0.303640  0.363878  0.266595     0.347969   
7.0   0.593680  0.353823  0.615776  0.404782  0.593157  0.389833     0.581174   
8.0   0.546720  0.350472  0.582676  0.395957  0.570217  0.388112     0.545209   
9.0   0.524636  0.335026  0.670749  0.474584  0.555788  0.376933     0.572858   
10.0  0.516722  0.306384  0.590200  0.390094  0.519328  0.316414     0.535502   
11.0  0.322961  0.242985  0.348294  0.263637  0.311792  0.228817     0.319871   
12.0  0.301490  0.211683  0.385191  0.345050  0.312546  0.218561     0.315443   
13.0  0.526347  0.313143  0.625358  0.403936  0.537806  0.319960     0.550766   
14.0  2.103598  2.072617  2.204908  1.801979  2.077678  1.898072     2.121873   
15.0  2.887670  2.392129  3.017366  2.010314  2.857847  2.234761     2.944444   
16.0  0.399605  0.366621  0.602107  0.497060  0.478710  0.367858     0.430044   
17.0  0.383683  0.272849  0.483291  0.322499  0.412647  0.362500     0.409392   
18.0  0.291353  0.198003  0.448164  0.363907  0.302495  0.200046     0.338582   
19.0  0.516833  0.327463  0.603649  0.415131  0.531108  0.330416     0.531479   
20.0  0.501107  0.307698  0.515970  0.319331  0.508101  0.307375     0.502258   

               xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
           std         mean       std         mean       std             mean   
모델                                                                              
0.0   0.327155     0.513160  0.324887     0.510289  0.332001         0.511677   
1.0   0.320621     0.529743  0.324014     0.535375  0.328721         0.529638   
2.0   0.222544     0.290328  0.194857     0.278697  0.229561         0.277067   
3.0   0.149208     0.282528  0.174687     0.307454  0.151378         0.296497   
4.0   2.909697     5.030646  3.059549     5.037767  2.881930         5.011913   
5.0   0.329162     0.477430  0.322015     0.476254  0.331647         0.472482   
6.0   0.280855     0.343832  0.265077     0.354978  0.273369         0.344013   
7.0   0.350619     0.583547  0.362869     0.582313  0.366211         0.575569   
8.0   0.348351     0.550878  0.358014     0.556259  0.353520         0.546044   
9.0   0.386457     0.532738  0.341400     0.583126  0.395285         0.556136   
10.0  0.329617     0.513187  0.307551     0.532994  0.329228         0.521130   
11.0  0.247195     0.313110  0.233540     0.311761  0.241361         0.310591   
12.0  0.247529     0.302187  0.207893     0.315504  0.252007         0.302539   
13.0  0.338293     0.526222  0.312328     0.553830  0.341875         0.535322   
14.0  1.894160     2.084583  1.970579     2.114564  1.832868         2.100554   
15.0  2.144942     2.870319  2.289589     2.928264  2.089766         2.912229   
16.0  0.338811     0.417799  0.334681     0.460765  0.332628         0.413190   
17.0  0.272348     0.390579  0.304061     0.419527  0.314781         0.397766   
18.0  0.244064     0.289434  0.191509     0.345529  0.247970         0.312433   
19.0  0.344489     0.512866  0.325643     0.541778  0.344633         0.522732   
20.0  0.304745     0.499047  0.305513     0.502187  0.301208         0.498998   

                
           std  
모델              
0.0   0.326344  
1.0   0.321449  
2.0   0.213367  
3.0   0.153072

In [62]:
# 'mean' 열만 필터링
mean_columns = [col for col in model_stats.columns if col[1] == 'mean']

# 스타일링 함수 정의
def highlight_minimum(s):
    is_min = s == s.min()  # 최소값 여부 확인
    return ['background-color: yellow' if v else '' for v in is_min]

# 스타일 적용
styled_model_stats = model_stats.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

styled_model_stats

In [65]:
model_stats_notnan = final_result_df[~final_result_df['배터리용량'].isna()].groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats_notnan

xgb-real            cat-real            lgb-real           xgb_cat-real  \
          mean       std      mean       std      mean       std         mean   
모델                                                                              
0.0   0.513365  0.330880  0.533132  0.369617  0.507227  0.346238     0.515512   
1.0   0.533638  0.320317  0.566821  0.355857  0.560161  0.335115     0.542959   
2.0   0.280929  0.187444  0.290661  0.268938  0.297065  0.222612     0.267014   
3.0   0.297286  0.180591  0.357495  0.166858  0.306114  0.185876     0.317924   
4.0   5.080060  3.256471  5.072983  2.886626  5.139987  3.127243     5.008884   
5.0   0.470526  0.315042  0.499478  0.345379  0.496535  0.330056     0.472916   
6.0   0.287620  0.231752  0.372912  0.317842  0.324192  0.224100     0.302549   
7.0   0.623153  0.360164  0.595663  0.404579  0.628069  0.403492     0.586020   
8.0   0.540448  0.352684  0.548087  0.354812  0.576967  0.393402     0.522968   
9.0   0.524312  0.337667  0.691877  0.492930  0.564367  0.395000     0.583641   
10.0  0.503214  0.301391  0.569755  0.379336  0.510806  0.318564     0.521266   
11.0  0.359852  0.271850  0.378322  0.276604  0.343500  0.255295     0.354260   
12.0  0.321751  0.236314  0.472998  0.486413  0.312449  0.226609     0.348964   
13.0  0.529729  0.314236  0.627155  0.407880  0.532885  0.327055     0.552499   
14.0  1.081480  1.595755  1.178092  1.395372  1.067070  1.477322     1.083206   
15.0  1.581094  1.503861  1.759539  1.464608  1.566562  1.440481     1.657398   
16.0  0.372466  0.375823  0.703154  0.529324  0.435754  0.354221     0.461941   
17.0  0.399474  0.282499  0.465208  0.320669  0.430199  0.382100     0.409518   
18.0  0.288778  0.179531  0.435364  0.353422  0.296041  0.192414     0.334100   
19.0  0.510796  0.312811  0.615469  0.429666  0.525882  0.323672     0.534223   
20.0  0.487235  0.317918  0.517668  0.348346  0.512537  0.315533     0.486607   

               xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
           std         mean       std         mean       std             mean   
모델                                                                              
0.0   0.341153     0.504089  0.338640     0.508932  0.354369         0.506999   
1.0   0.322506     0.540230  0.326381     0.551702  0.332032         0.542273   
2.0   0.225386     0.286367  0.198197     0.273226  0.238170         0.270770   
3.0   0.151155     0.293300  0.172243     0.317631  0.152502         0.306206   
4.0   2.954353     5.068089  3.141049     5.077316  2.942726         5.041491   
5.0   0.322291     0.478422  0.319973     0.478344  0.326553         0.471604   
6.0   0.268387     0.299938  0.219418     0.312766  0.257645         0.297993   
7.0   0.360455     0.617842  0.370193     0.594695  0.381390         0.593350   
8.0   0.330794     0.549743  0.361150     0.540560  0.334066         0.532457   
9.0   0.392859     0.538797  0.355656     0.601536  0.406401         0.567559   
10.0  0.321238     0.501631  0.307029     0.521079  0.324550         0.509910   
11.0  0.265532     0.347735  0.259688     0.344688  0.256825         0.344043   
12.0  0.293649     0.312397  0.219232     0.347183  0.298930         0.322363   
13.0  0.342597     0.528849  0.316280     0.554399  0.345711         0.537925   
14.0  1.461730     1.069343  1.528186     1.079921  1.423996         1.067313   
15.0  1.453818     1.570144  1.461236     1.647739  1.432494         1.621630   
16.0  0.357742     0.384084  0.333924     0.483865  0.357048         0.416920   
17.0  0.283111     0.406659  0.318638     0.420038  0.329287         0.403587   
18.0  0.228342     0.286751  0.183632     0.336762  0.232702         0.308393   
19.0  0.344426     0.514435  0.316581     0.540093  0.348316         0.524830   
20.0  0.321598     0.493310  0.316492     0.497737  0.321282         0.488193   

                
           std  
모델              
0.0   0.342927  
1.0   0.323018  
2.0   0.218545  
3.0   0.151610

In [66]:
model_stats_isnan = final_result_df[final_result_df['배터리용량'].isna()].groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats_isnan

xgb-real            cat-real            lgb-real           xgb_cat-real  \
          mean       std      mean       std      mean       std         mean   
모델                                                                              
0.0   0.524736  0.325086  0.519808  0.341314  0.511035  0.322202     0.516628   
1.0   0.515466  0.318708  0.516696  0.322929  0.516634  0.324825     0.510308   
2.0   0.295684  0.189682  0.297309  0.253326  0.302523  0.197050     0.284230   
3.0   0.264933  0.178339  0.326025  0.138274  0.271130  0.180992     0.291335   
4.0   5.009113  2.965788  4.974671  2.709850  4.882199  2.779431     4.952737   
5.0   0.485236  0.330451  0.495017  0.374903  0.470910  0.323127     0.481525   
6.0   0.412360  0.310939  0.429919  0.278748  0.424217  0.312287     0.417027   
7.0   0.473383  0.301058  0.697871  0.399222  0.450661  0.290527     0.561397   
8.0   0.569470  0.344803  0.708145  0.503099  0.545733  0.371011     0.625887   
9.0   0.525131  0.332573  0.638555  0.445563  0.542715  0.349049     0.556427   
10.0  0.616751  0.327827  0.741591  0.437975  0.582428  0.295982     0.640919   
11.0  0.267887  0.179240  0.303468  0.236943  0.264457  0.172581     0.268534   
12.0  0.293832  0.201566  0.352005  0.267141  0.312582  0.215899     0.302773   
13.0  0.510347  0.309906  0.616856  0.387776  0.561083  0.285248     0.542569   
14.0  3.818390  1.594948  3.927583  0.828663  3.773159  1.181950     3.864431   
15.0  4.932746  2.071676  4.986140  0.803835  4.878989  1.693505     4.958950   
16.0  0.460088  0.339187  0.376916  0.317931  0.574440  0.381152     0.358959   
17.0  0.284817  0.174165  0.596501  0.314053  0.302760  0.164109     0.408607   
18.0  0.304492  0.275275  0.513489  0.410465  0.335432  0.234314     0.361460   
19.0  0.542599  0.385234  0.553206  0.344430  0.553408  0.359362     0.519771   
20.0  0.504881  0.305375  0.515509  0.311686  0.506894  0.305726     0.506516   

               xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
           std         mean       std         mean       std             mean   
모델                                                                              
0.0   0.324573     0.515202  0.322323     0.510594  0.327446         0.512730   
1.0   0.317289     0.512758  0.320208     0.508932  0.322336         0.509174   
2.0   0.218521     0.296548  0.190142     0.287288  0.216042         0.286954   
3.0   0.145140     0.265425  0.177821     0.291294  0.148726         0.281081   
4.0   2.818360     4.917066  2.845621     4.917803  2.733707         4.922192   
5.0   0.345022     0.475294  0.327919     0.471753  0.343929         0.474373   
6.0   0.286625     0.410569  0.311902     0.419159  0.285205         0.413981   
7.0   0.309840     0.443571  0.295392     0.531770  0.294299         0.502994   
8.0   0.398979     0.554996  0.349878     0.613205  0.415309         0.595329   
9.0   0.377764     0.523505  0.319905     0.555073  0.377904         0.538729   
10.0  0.373647     0.598752  0.301347     0.621222  0.353700         0.604216   
11.0  0.207470     0.261420  0.176465     0.262604  0.207562         0.260651   
12.0  0.227054     0.298327  0.203757     0.303531  0.231358         0.295046   
13.0  0.319636     0.513799  0.295009     0.551134  0.325760         0.523007   
14.0  1.083393     3.787839  1.360948     3.850371  0.883152         3.834007   
15.0  1.355403     4.905375  1.822038     4.932564  1.182789         4.932296   
16.0  0.281005     0.492934  0.325522     0.409285  0.264757         0.404877   
17.0  0.194144     0.289903  0.156853     0.416323  0.204181         0.361321   
18.0  0.313714     0.303125  0.228875     0.390271  0.313230         0.333047   
19.0  0.346964     0.506167  0.364158     0.548970  0.330746         0.513777   
20.0  0.300499     0.500607  0.303063     0.503397  0.296149         0.501937   

                
           std  
모델              
0.0   0.323144  
1.0   0.318613  
2.0   0.205478  
3.0   0.154694

In [63]:
model_stats_notnan = final_result_df[~final_result_df['배터리용량'].isna()].groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats_isnan = final_result_df[final_result_df['배터리용량'].isna()].groupby('모델')[['xgb-real','cat-real','lgb-real','xgb_cat-real','xgb_lgb-real','cat_lgb-real','xgb_cat_lgb-real']].agg(['mean', 'std'])
model_stats_notnan - model_stats_isnan

xgb-real            cat-real            lgb-real           xgb_cat-real  \
          mean       std      mean       std      mean       std         mean   
모델                                                                              
0.0  -0.011370  0.005794  0.013325  0.028303 -0.003809  0.024036    -0.001117   
1.0   0.018171  0.001609  0.050126  0.032929  0.043526  0.010290     0.032651   
2.0  -0.014755 -0.002238 -0.006648  0.015612 -0.005458  0.025562    -0.017216   
3.0   0.032353  0.002252  0.031470  0.028584  0.034984  0.004883     0.026588   
4.0   0.070947  0.290682  0.098313  0.176776  0.257788  0.347812     0.056147   
5.0  -0.014710 -0.015410  0.004461 -0.029524  0.025625  0.006930    -0.008609   
6.0  -0.124741 -0.079187 -0.057007  0.039094 -0.100025 -0.088187    -0.114478   
7.0   0.149770  0.059107 -0.102208  0.005356  0.177408  0.112965     0.024622   
8.0  -0.029021  0.007882 -0.160058 -0.148287  0.031233  0.022390    -0.102919   
9.0  -0.000819  0.005094  0.053323  0.047367  0.021651  0.045951     0.027214   
10.0 -0.113537 -0.026435 -0.171837 -0.058639 -0.071622  0.022582    -0.119653   
11.0  0.091966  0.092610  0.074854  0.039661  0.079043  0.082714     0.085726   
12.0  0.027919  0.034748  0.120993  0.219272 -0.000133  0.010710     0.046191   
13.0  0.019382  0.004330  0.010299  0.020104 -0.028198  0.041807     0.009930   
14.0 -2.736911  0.000807 -2.749491  0.566709 -2.706089  0.295372    -2.781225   
15.0 -3.351652 -0.567815 -3.226601  0.660773 -3.312427 -0.253024    -3.301553   
16.0 -0.087622  0.036636  0.326239  0.211392 -0.138686 -0.026931     0.102982   
17.0  0.114657  0.108333 -0.131293  0.006616  0.127439  0.217991     0.000911   
18.0 -0.015714 -0.095744 -0.078126 -0.057043 -0.039391 -0.041901    -0.027360   
19.0 -0.031803 -0.072423  0.062263  0.085236 -0.027526 -0.035690     0.014451   
20.0 -0.017646  0.012543  0.002159  0.036660  0.005643  0.009807    -0.019909   

               xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
           std         mean       std         mean       std             mean   
모델                                                                              
0.0   0.016581    -0.011113  0.016317    -0.001662  0.026923        -0.005731   
1.0   0.005217     0.027472  0.006174     0.042770  0.009696         0.033099   
2.0   0.006864    -0.010180  0.008054    -0.014062  0.022128        -0.016184   
3.0   0.006016     0.027875 -0.005577     0.026337  0.003776         0.025125   
4.0   0.135992     0.151023  0.295428     0.159513  0.209019         0.119299   
5.0  -0.022731     0.003128 -0.007946     0.006591 -0.017376        -0.002769   
6.0  -0.018239    -0.110631 -0.092484    -0.106393 -0.027560        -0.115988   
7.0   0.050615     0.174271  0.074802     0.062925  0.087092         0.090355   
8.0  -0.068185    -0.005254  0.011272    -0.072645 -0.081243        -0.062872   
9.0   0.015095     0.015292  0.035752     0.046463  0.028497         0.028830   
10.0 -0.052409    -0.097120  0.005682    -0.100144 -0.029149        -0.094306   
11.0  0.058063     0.086315  0.083223     0.082084  0.049263         0.083392   
12.0  0.066595     0.014070  0.015475     0.043652  0.067571         0.027317   
13.0  0.022961     0.015050  0.021271     0.003265  0.019951         0.014918   
14.0  0.378337    -2.718496  0.167238    -2.770450  0.540844        -2.766694   
15.0  0.098415    -3.335230 -0.360802    -3.284825  0.249705        -3.310667   
16.0  0.076737    -0.108849  0.008403     0.074581  0.092291         0.012043   
17.0  0.088967     0.116756  0.161784     0.003715  0.125106         0.042266   
18.0 -0.085373    -0.016375 -0.045243    -0.053509 -0.080528        -0.024654   
19.0 -0.002537     0.008268 -0.047577    -0.008877  0.017570         0.011053   
20.0  0.021099    -0.007298  0.013429    -0.005660  0.025133        -0.013744   

                
           std  
모델              
0.0   0.019782  
1.0   0.004406  
2.0   0.013067  
3.0  -0.003084

In [72]:
import numpy as np

# Initialize lists to store RMSEs for each individual model
xgb_rmse_list = []
cat_rmse_list = []
lgb_rmse_list = []
voting_rmse_list = {key: [] for key in voting_models.keys()}

# Store the results of each iteration
model_stats_list = []
model_stats_isnan_list = []
model_stats_notnan_list = []

X_train_all = train.drop(columns=['ID', '가격(백만원)'])
y_train_all = train['가격(백만원)']
model_column_1 = train['모델']

# Repeat the process 10 times
for i in range(10):
    random_state = i  # Changing random state for each iteration

    # Stratified Split (with changing random_state)
    X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
        X_train_all, y_train_all, test_size=0.2, random_state=random_state, stratify=model_column_1
    )

    # Recreate and train the models with the new random_state
    xgb_model.fit(X_train_stratified, y_train_stratified)
    cat_model.fit(X_train_stratified, y_train_stratified)
    lgb_model.fit(X_train_stratified, y_train_stratified)

    # Predictions
    y_pred_xgb = xgb_model.predict(X_valid_stratified)
    y_pred_lgb = lgb_model.predict(X_valid_stratified)
    y_pred_cat = cat_model.predict(X_valid_stratified)

    # RMSE for individual models
    xgb_rmse = np.sqrt(((y_valid_stratified - y_pred_xgb) ** 2).mean())
    cat_rmse = np.sqrt(((y_valid_stratified - y_pred_cat) ** 2).mean())
    lgb_rmse = np.sqrt(((y_valid_stratified - y_pred_lgb) ** 2).mean())

    # Add the individual model RMSEs to the respective lists
    xgb_rmse_list.append(xgb_rmse)
    cat_rmse_list.append(cat_rmse)
    lgb_rmse_list.append(lgb_rmse)

    # Store RMSEs for voting models
    for name, voting_model in voting_models.items():
        y_pred_voting = voting_model.predict(X_valid_stratified)
        voting_rmse = np.sqrt(((y_valid_stratified - y_pred_voting) ** 2).mean())
        voting_rmse_list[name].append(voting_rmse)

    # Create result DataFrame
    result_df = pd.DataFrame(X_valid_stratified)
    result_df['가격(백만원)'] = y_valid_stratified.values
    result_df['XGB_Pred'] = y_pred_xgb
    result_df['CatBoost_Pred'] = y_pred_cat
    result_df['LGB_Pred'] = y_pred_lgb

    # Voting Model predictions
    for name, voting_model in voting_models.items():
        result_df[f'{name}_Pred'] = voting_model.predict(X_valid_stratified)

    # Compute errors
    result_df['xgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB_Pred'])**2)**0.5
    result_df['cat-real'] = ((result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
    result_df['lgb-real'] = ((result_df['가격(백만원)'] - result_df['LGB_Pred'])**2)**0.5
    result_df['xgb_cat-real'] = ((result_df['가격(백만원)'] - result_df['XGB+Cat_Pred'])**2)**0.5
    result_df['xgb_lgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB+LGB_Pred'])**2)**0.5
    result_df['cat_lgb-real'] = ((result_df['가격(백만원)'] - result_df['Cat+LGB_Pred'])**2)**0.5
    result_df['xgb_cat_lgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB+Cat+LGB_Pred'])**2)**0.5

    # Calculate model stats
    model_stats = result_df.groupby('모델')[['xgb-real', 'cat-real', 'lgb-real', 'xgb_cat-real', 'xgb_lgb-real', 'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    model_stats_notnan = result_df[~result_df['배터리용량'].isna()].groupby('모델')[['xgb-real', 'cat-real', 'lgb-real', 'xgb_cat-real', 'xgb_lgb-real', 'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    model_stats_isnan = result_df[result_df['배터리용량'].isna()].groupby('모델')[['xgb-real', 'cat-real', 'lgb-real', 'xgb_cat-real', 'xgb_lgb-real', 'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])

    # Append the stats to the lists
    model_stats_list.append(model_stats)
    model_stats_isnan_list.append(model_stats_isnan)
    model_stats_notnan_list.append(model_stats_notnan)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000888 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.335418
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000498 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.348574
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000889 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 404
[LightGBM] [Info] Number of data points in the train set: 599

In [73]:
# Calculate the average of each result
model_stats_avg = np.mean([ms.values for ms in model_stats_list], axis=0)
model_stats_isnan_avg = np.mean([ms_isnan.values for ms_isnan in model_stats_isnan_list], axis=0)
model_stats_notnan_avg = np.mean([ms_notnan.values for ms_notnan in model_stats_notnan_list], axis=0)

# Average RMSE for each individual model
xgb_rmse_avg = np.mean(xgb_rmse_list)
cat_rmse_avg = np.mean(cat_rmse_list)
lgb_rmse_avg = np.mean(lgb_rmse_list)

# Average RMSE for each voting model
voting_rmse_avg = {name: np.mean(rmses) for name, rmses in voting_rmse_list.items()}

# Convert them back to DataFrames for easier viewing, and transpose the data to match the correct shape
model_stats_avg_df = pd.DataFrame(model_stats_avg.T, index=model_stats.columns, columns=model_stats.index)
model_stats_isnan_avg_df = pd.DataFrame(model_stats_isnan_avg.T, index=model_stats_isnan.columns, columns=model_stats_isnan.index)
model_stats_notnan_avg_df = pd.DataFrame(model_stats_notnan_avg.T, index=model_stats_notnan.columns, columns=model_stats_notnan.index)

# Transpose the DataFrames to swap rows and columns
model_stats_avg_df = model_stats_avg_df.T
model_stats_isnan_avg_df = model_stats_isnan_avg_df.T
model_stats_notnan_avg_df = model_stats_notnan_avg_df.T

In [74]:
# Display the results
print("Average RMSE for individual models:")
print(f"XGB: {xgb_rmse_avg}")
print(f"CatBoost: {cat_rmse_avg}")
print(f"LGB: {lgb_rmse_avg}")
for name, rmse in voting_rmse_avg.items():
    print(f"{name}: {rmse}")

Average RMSE for individual models:
XGB: 1.4010632723943366
CatBoost: 1.3874213605440633
LGB: 1.3756917423202562
XGB+Cat: 1.0721222399570964
XGB+LGB: 1.00490406935706
Cat+LGB: 1.1424469168320286
XGB+Cat+LGB: 1.0633084135634656


In [75]:
print("\nTransposed average model stats:")
display(model_stats_avg_df)

print("\nTransposed average model stats (non-NaN battery capacity):")
display(model_stats_notnan_avg_df)

print("\nTransposed average model stats (NaN battery capacity):")
display(model_stats_isnan_avg_df)


Transposed average model stats:


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.526241  0.333338  0.526733  0.343001  0.523028  0.326757     0.463528   
1   0.542322  0.323944  0.564279  0.350904  0.554357  0.335379     0.506860   
2   0.288060  0.191311  0.299464  0.254842  0.301596  0.220849     0.260131   
3   0.281927  0.180931  0.348060  0.173699  0.290491  0.189687     0.284666   
4   4.794501  3.195353  4.699312  2.947413  4.840717  3.149835     2.756986   
5   0.491725  0.331012  0.507988  0.363912  0.509513  0.342959     0.462077   
6   0.334454  0.270245  0.408373  0.321889  0.360609  0.278306     0.333616   
7   0.592727  0.413562  0.604809  0.417747  0.594096  0.394078     0.491206   
8   0.531520  0.370252  0.580724  0.406606  0.562384  0.433737     0.441722   
9   0.521200  0.341963  0.647993  0.470850  0.545175  0.376855     0.505939   
10  0.527300  0.308547  0.606135  0.408427  0.523068  0.321780     0.489577   
11  0.316997  0.252994  0.350855  0.276365  0.306952  0.237899     0.284913   
12  0.301826  0.226911  0.393920  0.370934  0.305642  0.224022     0.283215   
13  0.528787  0.313061  0.623870  0.414817  0.543570  0.333691     0.497945   
14  2.135035  2.130920  2.219452  1.796836  2.110098  1.941674     1.832343   
15  2.904936  2.434854  3.048326  2.028712  2.858320  2.251622     2.518081   
16  0.396599  0.399022  0.588245  0.513623  0.477917  0.412976     0.393235   
17  0.380210  0.274221  0.484803  0.362713  0.404003  0.346628     0.359987   
18  0.303417  0.215712  0.451966  0.359722  0.312264  0.209868     0.308387   
19  0.511954  0.327069  0.616184  0.405905  0.526299  0.335571     0.498803   
20  0.508547  0.321833  0.533484  0.342871  0.520464  0.320951     0.468672   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.290030     0.446031  0.284233     0.492503  0.308811         0.466816   
1   0.302519     0.477515  0.292304     0.507667  0.312689         0.495107   
2   0.233667     0.258092  0.193535     0.267464  0.237127         0.259686   
3   0.133945     0.246335  0.153908     0.283486  0.140361         0.269317   
4   1.950798     2.373391  1.906903     3.271207  2.104068         2.793743   
5   0.319922     0.443840  0.296989     0.465503  0.319622         0.453956   
6   0.252550     0.300766  0.226062     0.337053  0.246565         0.317481   
7   0.326871     0.449564  0.292894     0.503042  0.327508         0.477225   
8   0.298019     0.417511  0.288729     0.484348  0.321891         0.443969   
9   0.367423     0.441725  0.281653     0.514952  0.351283         0.480533   
10  0.293797     0.444875  0.264941     0.495421  0.308352         0.470377   
11  0.224425     0.264442  0.195212     0.292370  0.232460         0.275644   
12  0.227305     0.253918  0.175629     0.290886  0.230957         0.264209   
13  0.320384     0.457921  0.280144     0.508031  0.325720         0.480063   
14  1.553511     1.668753  1.569579     1.919814  1.611352         1.800282   
15  1.825575     2.287045  1.895527     2.679602  1.914739         2.492554   
16  0.298851     0.319884  0.263864     0.415619  0.319792         0.351551   
17  0.247997     0.316488  0.226797     0.374645  0.260766         0.339125   
18  0.250362     0.255519  0.180035     0.312387  0.250538         0.283530   
19  0.329146     0.441619  0.291932     0.506568  0.335784         0.473336   
20  0.275760     0.451784  0.276603     0.476605  0.280370         0.464329   

              
         std  
모델            
0   0.290151  
1   0.299373  
2   0.217937  
3   0.136722  
4   1.939176  
5   0.308089  
6   0.235308  
7   0.303771  
8   0.295608  
9   0.323233  
10  0.285123  



Transposed average model stats (non-NaN battery capacity):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.493305  0.333272  0.507649  0.356680  0.503879  0.350704     0.411898   
1   0.548029  0.329689  0.575622  0.367560  0.566595  0.339178     0.505392   
2   0.282380  0.189248  0.298229  0.262180  0.300052  0.230318     0.253680   
3   0.293913  0.179998  0.352074  0.181936  0.303120  0.191810     0.290821   
4   4.964297  3.250572  4.908957  2.984265  5.068569  3.229703     2.727740   
5   0.475073  0.331053  0.494725  0.355740  0.504926  0.353485     0.450239   
6   0.283283  0.220311  0.390146  0.334447  0.319227  0.230595     0.294864   
7   0.620488  0.428737  0.585158  0.412612  0.626362  0.401063     0.496864   
8   0.531579  0.372568  0.535349  0.337193  0.567959  0.442498     0.428938   
9   0.528452  0.341927  0.661518  0.473808  0.560260  0.391011     0.497630   
10  0.512628  0.304036  0.578292  0.397116  0.513015  0.323006     0.482763   
11  0.351121  0.284103  0.386244  0.284508  0.338320  0.263450     0.305350   
12  0.306144  0.249699  0.501488  0.520830  0.299547  0.235454     0.300419   
13  0.533905  0.309807  0.625372  0.420326  0.541131  0.334503     0.503238   
14  1.120589  1.661108  1.198864  1.390178  1.102901  1.514107     0.947465   
15  1.607114  1.530029  1.825863  1.477184  1.601558  1.466544     1.446470   
16  0.367713  0.386012  0.693378  0.552915  0.427042  0.386666     0.431968   
17  0.398776  0.285279  0.454728  0.324455  0.425016  0.365250     0.356732   
18  0.299869  0.190101  0.434738  0.348050  0.307475  0.206113     0.315920   
19  0.508237  0.315073  0.611508  0.416084  0.525023  0.327391     0.507475   
20  0.478918  0.330160  0.529438  0.339706  0.520803  0.332161     0.433646   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.268976     0.362694  0.264626     0.462560  0.281742         0.410969   
1   0.308675     0.473408  0.293043     0.512470  0.323271         0.494091   
2   0.236056     0.250216  0.195618     0.257907  0.240382         0.250958   
3   0.134642     0.247671  0.153070     0.286136  0.142236         0.271992   
4   1.933525     2.366499  1.916147     3.381356  2.101864         2.816291   
5   0.313309     0.430539  0.295666     0.452041  0.311558         0.440335   
6   0.245185     0.247278  0.186793     0.299277  0.231130         0.269961   
7   0.325043     0.471255  0.295880     0.514523  0.325676         0.490333   
8   0.267563     0.413732  0.271646     0.475503  0.298583         0.436117   
9   0.354908     0.443074  0.285867     0.511837  0.349230         0.477210   
10  0.294700     0.438248  0.268575     0.484649  0.306862         0.462676   
11  0.231841     0.286207  0.210789     0.322847  0.238990         0.298895   
12  0.238923     0.252107  0.171295     0.317777  0.245215         0.263437   
13  0.326075     0.466934  0.284583     0.512317  0.330537         0.486981   
14  1.103155     0.829302  1.077766     0.986732  1.184738         0.910605   
15  1.165789     1.250847  1.094274     1.537784  1.254295         1.407851   
16  0.324018     0.300031  0.257624     0.449369  0.338587         0.363998   
17  0.249138     0.329024  0.238961     0.372374  0.257877         0.341657   
18  0.247488     0.254069  0.171993     0.318956  0.249440         0.287969   
19  0.334740     0.449265  0.286206     0.517063  0.343654         0.484517   
20  0.278151     0.419438  0.288802     0.442021  0.284865         0.429128   

              
         std  
모델            
0   0.264128  
1   0.304484  
2   0.221021  
3   0.135166  
4   1.935181  
5   0.302734  
6   0.212022  
7   0.304668  
8   0.271785  
9   0.318583  
10  0.285580  



Transposed average model stats (NaN battery capacity):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.533317  0.332653  0.530749  0.334343  0.527462  0.321074     0.475020   
1   0.533256  0.316745  0.545788  0.321529  0.535080  0.329604     0.509023   
2   0.298028  0.195393  0.301922  0.238589  0.305135  0.207120     0.271740   
3   0.265962  0.178135  0.342314  0.162329  0.273199  0.185113     0.276128   
4   4.587493  2.833633  4.243904  2.696529  4.347020  2.791342     2.877922   
5   0.530848  0.331162  0.536903  0.377994  0.521531  0.321813     0.489397   
6   0.406668  0.310963  0.424169  0.281509  0.418086  0.322758     0.388658   
7   0.466151  0.302047  0.707749  0.404519  0.459603  0.335692     0.468190   
8   0.533789  0.374216  0.759239  0.558352  0.535262  0.398910     0.485549   
9   0.511192  0.343845  0.627184  0.459737  0.522398  0.346340     0.517454   
10  0.620411  0.324095  0.783570  0.410264  0.581865  0.314741     0.526057   
11  0.268513  0.187637  0.300470  0.254678  0.262988  0.185964     0.255916   
12  0.299559  0.211832  0.351351  0.275499  0.309426  0.219720     0.277439   
13  0.505000  0.328611  0.611707  0.380015  0.554706  0.335807     0.469187   
14  3.842786  1.703224  3.918927  0.858597  3.801456  1.299581     3.304160   
15  5.008427  2.154904  5.035786  0.857623  4.898878  1.720283     4.256994   
16  0.453217  0.379614  0.371711  0.324342  0.580398  0.422508     0.311237   
17  0.264672  0.157113  0.628350  0.461724  0.277949  0.154605     0.374330   
18  0.309564  0.289781  0.542281  0.365159  0.326573  0.204899     0.253641   
19  0.532445  0.361886  0.639923  0.359364  0.537576  0.354201     0.471025   
20  0.518127  0.319481  0.536628  0.341386  0.521813  0.318375     0.479789   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.291748     0.464688  0.285396     0.499319  0.312102         0.479323   
1   0.294505     0.484042  0.293095     0.499677  0.297367         0.496584   
2   0.230706     0.273973  0.192660     0.284189  0.233242         0.275616   
3   0.133337     0.244708  0.154407     0.279858  0.137850         0.265668   
4   1.930965     2.498449  1.923152     3.012183  2.053397         2.791912   
5   0.334953     0.474455  0.299422     0.496765  0.336218         0.485447   
6   0.248494     0.378877  0.254274     0.388620  0.254221         0.385061   
7   0.314259     0.353041  0.265234     0.452658  0.318048         0.420715   
8   0.385226     0.427074  0.351915     0.515577  0.391518         0.469520   
9   0.386554     0.438262  0.279298     0.517635  0.357267         0.483841   
10  0.292879     0.484968  0.242932     0.557258  0.318292         0.514307   
11  0.207863     0.233248  0.165235     0.250454  0.213867         0.243020   
12  0.222605     0.255141  0.176285     0.280543  0.224152         0.265369   
13  0.290965     0.418250  0.262601     0.483685  0.306182         0.445964   
14  0.959835     3.068248  1.224330     3.470496  0.848569         3.280968   
15  1.277530     3.959660  1.702196     4.526760  1.220116         4.247805   
16  0.216365     0.357269  0.268390     0.342873  0.261719         0.321953   
17  0.243913     0.239606  0.110428     0.384279  0.270794         0.320813   
18  0.254579     0.250986  0.215676     0.263399  0.245677         0.246463   
19  0.299389     0.416769  0.307407     0.471298  0.290867         0.434629   
20  0.273992     0.461527  0.272961     0.487252  0.277930         0.475183   

              
         std  
모델            
0   0.293151  
1   0.293154  
2   0.214538  
3   0.139091  
4   1.934749  
5   0.319483  
6   0.248153  
7   0.285743  
8   0.372055  
9   0.334331  
10  0.286529  


In [79]:
# 'mean' 열만 필터링
mean_columns = [col for col in model_stats.columns if col[1] == 'mean']

# 스타일링 함수 정의
def highlight_minimum(s):
    is_min = s == s.min()  # 최소값 여부 확인
    return ['background-color: yellow' if v else '' for v in is_min]

# 스타일 적용
styled_model_stats_avg_df = model_stats_avg_df.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

styled_model_stats_notnan_avg_df = model_stats_notnan_avg_df.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

styled_model_stats_isnan_avg_df = model_stats_isnan_avg_df.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

# Displaying the styled DataFrames
styled_model_stats_avg_df, styled_model_stats_notnan_avg_df, styled_model_stats_isnan_avg_df

def highlight_max(s):
    is_max = s == s.max()  # Find maximum values
    return ['background-color: yellow' if v else '' for v in is_max]

# Subtract the DataFrames and apply styling
styled_model_stats_isnan_notnan_avg_df = (model_stats_isnan_avg_df - model_stats_notnan_avg_df).style.apply(
    highlight_max,
    subset=mean_columns,
    axis=1  # Apply row-wise
)
styled_model_stats_isnan_notnan_avg_df

In [77]:
display(styled_model_stats_avg_df)
display(styled_model_stats_notnan_avg_df)
display(styled_model_stats_isnan_avg_df)

In [ ]:
# 가격 데이터에 루트를 씌움

In [95]:
import numpy as np

# Initialize lists to store RMSEs for each individual model
xgb_rmse_list = []
cat_rmse_list = []
lgb_rmse_list = []
voting_rmse_list = {key: [] for key in voting_models.keys()}

# Store the results of each iteration
model_stats_list = []
model_stats_isnan_list = []
model_stats_notnan_list = []

train.loc[train['모델'] == 4, '가격(백만원)'] = np.sqrt(train.loc[train['모델'] == 4, '가격(백만원)'])

X_train_all = train.drop(columns=['ID', '가격(백만원)'])
y_train_all = train['가격(백만원)']
model_column_1 = train['모델']

# Repeat the process 10 times
for i in range(10):
    random_state = i  # Changing random state for each iteration

    # Stratified Split (with changing random_state)
    X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
        X_train_all, y_train_all, test_size=0.2, random_state=random_state, stratify=model_column_1
    )

    # Recreate and train the models with the new random_state
    xgb_model.fit(X_train_stratified, y_train_stratified)
    cat_model.fit(X_train_stratified, y_train_stratified)
    lgb_model.fit(X_train_stratified, y_train_stratified)

    # Predictions
    y_pred_xgb = xgb_model.predict(X_valid_stratified)
    y_pred_lgb = lgb_model.predict(X_valid_stratified)
    y_pred_cat = cat_model.predict(X_valid_stratified)

    # RMSE for individual models
    xgb_rmse = np.sqrt(((y_valid_stratified - y_pred_xgb) ** 2).mean())
    cat_rmse = np.sqrt(((y_valid_stratified - y_pred_cat) ** 2).mean())
    lgb_rmse = np.sqrt(((y_valid_stratified - y_pred_lgb) ** 2).mean())

    # Add the individual model RMSEs to the respective lists
    xgb_rmse_list.append(xgb_rmse)
    cat_rmse_list.append(cat_rmse)
    lgb_rmse_list.append(lgb_rmse)

    # Store RMSEs for voting models
    for name, voting_model in voting_models.items():
        y_pred_voting = voting_model.predict(X_valid_stratified)
        voting_rmse = np.sqrt(((y_valid_stratified - y_pred_voting) ** 2).mean())
        voting_rmse_list[name].append(voting_rmse)

    # Create result DataFrame
    result_df = pd.DataFrame(X_valid_stratified)
    result_df['가격(백만원)'] = y_valid_stratified.values
    result_df.loc[result_df['모델'] == 4, '가격(백만원)'] = result_df.loc[result_df['모델'] == 4, '가격(백만원)'] ** 2

    result_df['XGB_Pred'] = y_pred_xgb
    result_df['CatBoost_Pred'] = y_pred_cat
    result_df['LGB_Pred'] = y_pred_lgb

    # Voting Model predictions
    for name, voting_model in voting_models.items():
        result_df[f'{name}_Pred'] = voting_model.predict(X_valid_stratified)

    # Compute errors
    result_df['xgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB_Pred'])**2)**0.5
    result_df['cat-real'] = ((result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
    result_df['lgb-real'] = ((result_df['가격(백만원)'] - result_df['LGB_Pred'])**2)**0.5
    result_df['xgb_cat-real'] = ((result_df['가격(백만원)'] - result_df['XGB+Cat_Pred'])**2)**0.5
    result_df['xgb_lgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB+LGB_Pred'])**2)**0.5
    result_df['cat_lgb-real'] = ((result_df['가격(백만원)'] - result_df['Cat+LGB_Pred'])**2)**0.5
    result_df['xgb_cat_lgb-real'] = ((result_df['가격(백만원)'] - result_df['XGB+Cat+LGB_Pred'])**2)**0.5

    # Calculate model stats
    model_stats = result_df.groupby('모델')[['xgb-real', 'cat-real', 'lgb-real', 'xgb_cat-real', 'xgb_lgb-real', 'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    model_stats_notnan = result_df[~result_df['배터리용량'].isna()].groupby('모델')[['xgb-real', 'cat-real', 'lgb-real', 'xgb_cat-real', 'xgb_lgb-real', 'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    model_stats_isnan = result_df[result_df['배터리용량'].isna()].groupby('모델')[['xgb-real', 'cat-real', 'lgb-real', 'xgb_cat-real', 'xgb_lgb-real', 'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])

    # Append the stats to the lists
    model_stats_list.append(model_stats)
    model_stats_isnan_list.append(model_stats_isnan)
    model_stats_notnan_list.append(model_stats_notnan)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.073849
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000698 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.088826
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000887 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4

In [96]:
# Calculate the average of each result
model_stats_avg = np.mean([ms.values for ms in model_stats_list], axis=0)
model_stats_isnan_avg = np.mean([ms_isnan.values for ms_isnan in model_stats_isnan_list], axis=0)
model_stats_notnan_avg = np.mean([ms_notnan.values for ms_notnan in model_stats_notnan_list], axis=0)

# Average RMSE for each individual model
xgb_rmse_avg = np.mean(xgb_rmse_list)
cat_rmse_avg = np.mean(cat_rmse_list)
lgb_rmse_avg = np.mean(lgb_rmse_list)

# Average RMSE for each voting model
voting_rmse_avg = {name: np.mean(rmses) for name, rmses in voting_rmse_list.items()}

# Convert them back to DataFrames for easier viewing, and transpose the data to match the correct shape
model_stats_avg_df = pd.DataFrame(model_stats_avg.T, index=model_stats.columns, columns=model_stats.index)
model_stats_isnan_avg_df = pd.DataFrame(model_stats_isnan_avg.T, index=model_stats_isnan.columns, columns=model_stats_isnan.index)
model_stats_notnan_avg_df = pd.DataFrame(model_stats_notnan_avg.T, index=model_stats_notnan.columns, columns=model_stats_notnan.index)

# Transpose the DataFrames to swap rows and columns
model_stats_avg_df = model_stats_avg_df.T
model_stats_isnan_avg_df = model_stats_isnan_avg_df.T
model_stats_notnan_avg_df = model_stats_notnan_avg_df.T

In [97]:
# Display the results
print("Average RMSE for individual models:")
print(f"XGB: {xgb_rmse_avg}")
print(f"CatBoost: {cat_rmse_avg}")
print(f"LGB: {lgb_rmse_avg}")
for name, rmse in voting_rmse_avg.items():
    print(f"{name}: {rmse}")

Average RMSE for individual models:
XGB: 1.173752020897263
CatBoost: 1.1677041155269268
LGB: 1.1322357577571218
XGB+Cat: 2.181789575514265
XGB+LGB: 2.1496023188895834
Cat+LGB: 2.1943065911721353
XGB+Cat+LGB: 2.1697491206078254


In [99]:
# 'mean' 열만 필터링
mean_columns = [col for col in model_stats.columns if col[1] == 'mean']

# 스타일링 함수 정의
def highlight_minimum(s):
    is_min = s == s.min()  # 최소값 여부 확인
    return ['background-color: yellow' if v else '' for v in is_min]

# 스타일 적용
styled_model_stats_avg_df = model_stats_avg_df.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

styled_model_stats_notnan_avg_df = model_stats_notnan_avg_df.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

styled_model_stats_isnan_avg_df = model_stats_isnan_avg_df.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
)

# Displaying the styled DataFrames
styled_model_stats_avg_df, styled_model_stats_notnan_avg_df, styled_model_stats_isnan_avg_df

def highlight_max(s):
    is_max = s == s.max()  # Find maximum values
    return ['background-color: yellow' if v else '' for v in is_max]

# Subtract the DataFrames and apply styling
styled_model_stats_isnan_notnan_avg_df = (model_stats_isnan_avg_df - model_stats_notnan_avg_df).style.apply(
    highlight_max,
    subset=mean_columns,
    axis=1  # Apply row-wise
)

In [100]:
display(styled_model_stats_avg_df)
display(styled_model_stats_notnan_avg_df)
display(styled_model_stats_isnan_avg_df)